In [1]:
import os

def mostrar_arbol(ruta, nivel=0):
    indent = "│   " * nivel
    print(f"{indent}📁 {os.path.basename(ruta)}/")

    try:
        for elemento in os.listdir(ruta):
            path = os.path.join(ruta, elemento)
            if os.path.isdir(path):
                mostrar_arbol(path, nivel + 1)
            else:
                print(f"{'│   ' * (nivel + 1)}📄 {elemento}")
    except PermissionError:
        print(f"{'│   ' * (nivel + 1)}⛔ Permiso denegado")

# Cambia '.' por la ruta raíz de tu proyecto
mostrar_arbol(".")


📁 ./
│   📄 CNCTECHO_2024-2025 (2).kmz
│   📁 Comunas/
│   │   📄 comunas.CPG
│   │   📄 comunas.dbf
│   │   📄 comunas.prj
│   │   📄 comunas.sbn
│   │   📄 comunas.sbx
│   │   📄 comunas.shp
│   │   📄 comunas.shp.xml
│   │   📄 comunas.shx
│   📁 data/
│   │   📄 campamentos_poligonos.geojson
│   │   📄 campamentos_puntos.geojson
│   │   📄 comunas.geojson
│   │   📄 comunas_base.geojson
│   │   📄 regiones.geojson
│   │   📄 regiones_base.geojson
│   📄 entrega_2.ipynb
│   📄 index.html
│   📁 Regiones/
│   │   📄 Regional.CPG
│   │   📄 Regional.dbf
│   │   📄 Regional.prj
│   │   📄 Regional.sbn
│   │   📄 Regional.sbx
│   │   📄 Regional.shp
│   │   📄 Regional.shp.xml
│   │   📄 Regional.shx
│   📁 tmp_kmz_extract/
│   │   📄 doc.kml


In [2]:
# 0) Instalar (si hace falta) y preparar carpeta de salida
# (Si estás en Jupyter, puedes ejecutar estas líneas)

# !pip install geopandas shapely fiona pyogrio

import os
os.makedirs("data", exist_ok=True)


In [1]:
import geopandas as gpd

# 1) Leer shapefiles originales
gdf_regiones = gpd.read_file("Regiones/Regional.shp")
gdf_comunas  = gpd.read_file("Comunas/comunas.shp")

# 2) Asegurar CRS WGS84 (EPSG:4326) para la web
if gdf_regiones.crs is None or gdf_regiones.crs.to_epsg() != 4326:
    gdf_regiones = gdf_regiones.to_crs(4326)
if gdf_comunas.crs is None or gdf_comunas.crs.to_epsg() != 4326:
    gdf_comunas = gdf_comunas.to_crs(4326)

# 3) (Opcional) Simplificar geometría para web (tolerancia ~200m en métrico)
#    Simplificamos en 3857 (metros) y volvemos a 4326
def simplify_for_web(gdf, tol_m=200):
    gdf_m = gdf.to_crs(3857)
    gdf_m["geometry"] = gdf_m.geometry.simplify(tol_m, preserve_topology=True)
    return gdf_m.to_crs(4326)

gdf_regiones_s = simplify_for_web(gdf_regiones, tol_m=200)
gdf_comunas_s  = simplify_for_web(gdf_comunas,  tol_m=120)

# 4) Guardar GeoJSON base (sin datos aún)
gdf_regiones_s.to_file("data/regiones_base.geojson", driver="GeoJSON")
gdf_comunas_s.to_file("data/comunas_base.geojson",  driver="GeoJSON")

print("Listo: data/regiones_base.geojson, data/comunas_base.geojson")


Listo: data/regiones_base.geojson, data/comunas_base.geojson


In [ ]:
import zipfile, os
from pathlib import Path
import geopandas as gpd
import fiona

kmz_path = "CNCTECHO_2024-2025 (2).kmz"
work_dir = Path("tmp_kmz_extract")
work_dir.mkdir(exist_ok=True)

# 1) Descomprimir KMZ → KML(s)
with zipfile.ZipFile(kmz_path, "r") as z:
    z.extractall(work_dir)

# 2) Buscar el primer .kml dentro
kml_files = list(work_dir.rglob("*.kml"))
if not kml_files:
    raise FileNotFoundError("No se encontró ningún .kml dentro del KMZ.")
kml_path = str(kml_files[0])
print("KML encontrado:", kml_path)

# 3) Inspeccionar capas del KML (algunos KML tienen varias 'layers')
layers = fiona.listlayers(kml_path)
print("Capas disponibles en el KML:", layers)

# 4) Leer todas las capas y concatenar (si hay varias)
gdfs = []
for lyr in layers:
    try:
        gdf = gpd.read_file(kml_path, layer=lyr)
        if not gdf.empty:
            gdfs.append(gdf)
    except Exception as e:
        print(f"Problema leyendo capa {lyr}: {e}")

if not gdfs:
    raise RuntimeError("No fue posible leer ninguna capa con geometrías desde el KML.")

gdf_campamentos = gpd.pd.concat(gdfs, ignore_index=True)

# 5) Asegurar CRS WGS84
if gdf_campamentos.crs is None or gdf_campamentos.crs.to_epsg() != 4326:
    gdf_campamentos = gdf_campamentos.set_crs(4326, allow_override=True)

# Filtrar solo puntos (por si hay otros tipos)
gdf_campamentos = gdf_campamentos[gdf_campamentos.geometry.geom_type == "Point"].copy()

print("Total de features (puntos) de campamentos:", len(gdf_campamentos))
gdf_campamentos.to_file("data/campamentos_puntos.geojson", driver="GeoJSON")
print("Guardado: data/campamentos_puntos.geojson")


KML encontrado: tmp_kmz_extract\doc.kml
Capas disponibles en el KML: ['CNC_2024-2025']
Total de features (puntos) de campamentos: 0
Guardado: data/campamentos_puntos.geojson


In [ ]:
import geopandas as gpd

kml_path = "tmp_kmz_extract/doc.kml"
gdf = gpd.read_file(kml_path, layer="CNC_2024-2025")  # la capa que te listó fiona
print("Total features:", len(gdf))
print("Tipos geométricos:\n", gdf.geom_type.value_counts(dropna=False))
print("Columnas:", list(gdf.columns))
gdf.head(3)


Total features: 1428
Tipos geométricos:
 MultiPolygon    1428
Name: count, dtype: int64
Columnas: ['Name', 'Description', 'geometry']


,Name,Description,geometry
0,Lomas de Ramón Ángel Jara,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...","MULTIPOLYGON Z (((-71.40614 -33.07103 0, -71.4..."
1,Pompeya Sur,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...","MULTIPOLYGON Z (((-71.47948 -33.06025 0, -71.4..."
2,Mirador troncal sur,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...","MULTIPOLYGON Z (((-71.46351 -33.05897 0, -71.4..."


In [ ]:
import geopandas as gpd

# Carga del KML que leíste antes
kml_path = "tmp_kmz_extract/doc.kml"
layer_name = "CNC_2024-2025"

gdf_poly = gpd.read_file(kml_path, layer=layer_name)

# Asegurar CRS WGS84
if gdf_poly.crs is None or gdf_poly.crs.to_epsg() != 4326:
    gdf_poly = gdf_poly.set_crs(4326, allow_override=True)

# (Opcional robustez) Arreglar geometrías inválidas
gdf_poly["geometry"] = gdf_poly.geometry.buffer(0)

# Generar un punto dentro del polígono (mejor que centroid)
gdf_pts = gdf_poly.copy()
gdf_pts["geometry"] = gdf_pts.geometry.representative_point()

print("Features originales:", len(gdf_poly))
print("Puntos generados:", len(gdf_pts))
gdf_pts.to_file("data/campamentos_puntos.geojson", driver="GeoJSON")
print("Guardado: data/campamentos_puntos.geojson")


Features originales: 1428
Puntos generados: 1428
Guardado: data/campamentos_puntos.geojson


In [ ]:
import geopandas as gpd
import pandas as pd

reg = gpd.read_file("data/regiones_base.geojson").to_crs(4326)
com = gpd.read_file("data/comunas_base.geojson").to_crs(4326)
pts = gpd.read_file("data/campamentos_puntos.geojson").to_crs(4326)

# -----------------------
# 1) ASIGNACIÓN A REGIONES (1 punto -> 1 región)
# -----------------------
# Join con puntos a la izquierda
jr = gpd.sjoin(pts, reg, how="left", predicate="within")

# Algunos puntos pueden quedar sin región (NaN en index_right) por simplificación/bordes.
no_reg = jr["index_right"].isna().sum()
if no_reg > 0:
    # Plan B: reintenta con intersects sólo para los no asignados
    missing_pts = pts.loc[jr[jr["index_right"].isna()].index]
    jr_retry = gpd.sjoin(missing_pts, reg, how="left", predicate="intersects")
    # Combinar resultados: donde había NaN, usa lo del retry
    jr.loc[jr["index_right"].isna(), "index_right"] = jr_retry["index_right"]

# Si aún quedan NaN, último recurso: región más cercana (en CRS proyectado)
still_na = jr["index_right"].isna().sum()
if still_na > 0:
    # Convertir a CRS proyectado para distancias correctas
    pts_proj = pts.to_crs(3857)
    reg_proj = reg.to_crs(3857)
    missing_idx = jr[jr["index_right"].isna()].index
    nearest = gpd.sjoin_nearest(pts_proj.loc[missing_idx], reg_proj, how="left", max_distance=None)
    jr.loc[jr["index_right"].isna(), "index_right"] = nearest["index_right"]

# Conteo por región (1-1 garantizado)
conteo_reg = (
    jr["index_right"]
    .value_counts()
    .reindex(reg.index, fill_value=0)
    .rename("campamentos")
    .astype(int)
)
reg_out = reg.join(conteo_reg)

# -----------------------
# 2) ASIGNACIÓN A COMUNAS (1 punto -> 1 comuna)
# -----------------------
jc = gpd.sjoin(pts, com, how="left", predicate="within")

# Si un punto cae en varias comunas (solapes), quedará repetido.
# Nos quedamos con UNA comuna por punto: la "primera" fila por índice del punto.
jc = jc.reset_index()  # trae 'index' = índice original del punto
jc = jc.drop_duplicates(subset="index", keep="first")  # 1 fila por punto

# Completar no asignados con intersects
no_com = jc["index_right"].isna().sum()
if no_com > 0:
    missing_pts = pts.loc[jc.loc[jc["index_right"].isna(), "index"]]
    jc_retry = gpd.sjoin(missing_pts, com, how="left", predicate="intersects")
    # Crear un mapa del índice del punto al índice de la comuna
    map_retry = pd.Series(jc_retry["index_right"].values, index=jc_retry.index)
    jc.loc[jc["index_right"].isna(), "index_right"] = jc.loc[jc["index_right"].isna(), "index"].map(map_retry)

# Si aún quedan NaN, usar la comuna más cercana (en CRS proyectado)
still_na_c = jc["index_right"].isna().sum()
if still_na_c > 0:
    # Convertir a CRS proyectado para distancias correctas
    pts_proj = pts.to_crs(3857)
    com_proj = com.to_crs(3857)
    missing_idx = jc.loc[jc["index_right"].isna(), "index"].values
    nearest_c = gpd.sjoin_nearest(pts_proj.loc[missing_idx], com_proj, how="left", max_distance=None)
    map_near = pd.Series(nearest_c["index_right"].values, index=nearest_c.index)
    jc.loc[jc["index_right"].isna(), "index_right"] = jc.loc[jc["index_right"].isna(), "index"].map(map_near)

# Conteo por comuna (1-1 garantizado)
conteo_com = (
    jc["index_right"]
    .value_counts()
    .reindex(com.index, fill_value=0)
    .rename("campamentos")
    .astype(int)
)
com_out = com.join(conteo_com)

# -----------------------
# 3) Guardar y checks
# -----------------------
reg_out.to_file("data/regiones.geojson", driver="GeoJSON")
com_out.to_file("data/comunas.geojson",  driver="GeoJSON")

print("Total puntos:", len(pts))
print("Suma regiones:", reg_out["campamentos"].sum())
print("Suma comunas :", com_out["campamentos"].sum())

Total puntos: 1428
Suma regiones: 1428
Suma comunas : 1428


# Verificación: Todas las comunas de la Región Metropolitana

In [ ]:
# Importar librería necesaria
import geopandas as gpd

# Cargar el archivo de comunas generado
comunas_gdf = gpd.read_file("data/comunas.geojson")

# Filtrar solo las comunas de la Región Metropolitana
rm_comunas = comunas_gdf[comunas_gdf['Region'].str.contains('Metropolitana', na=False)].copy()

# Ordenar por cantidad de campamentos (descendente)
rm_comunas_sorted = rm_comunas.sort_values('campamentos', ascending=False)

# Mostrar información
print(f"╔══════════════════════════════════════════════════════════════╗")
print(f"║  REGIÓN METROPOLITANA DE SANTIAGO - TODAS LAS COMUNAS        ║")
print(f"╚══════════════════════════════════════════════════════════════╝\n")

print(f"Total de comunas: {len(rm_comunas)}")
print(f"Comunas CON campamentos: {len(rm_comunas[rm_comunas['campamentos'] > 0])}")
print(f"Comunas SIN campamentos (0): {len(rm_comunas[rm_comunas['campamentos'] == 0])}")
print(f"Total de campamentos: {rm_comunas['campamentos'].sum()}\n")

print("="*70)
print(f"{'COMUNA':<30} {'CAMPAMENTOS':>15}")
print("="*70)

# Mostrar TODAS las comunas (con y sin campamentos)
for idx, row in rm_comunas_sorted.iterrows():
    camp = row['campamentos']
    comuna = row['Comuna']
    
    # Marcar visualmente las que tienen campamentos
    if camp > 0:
        print(f"{comuna:<30} {camp:>15} 🔴")
    else:
        print(f"{comuna:<30} {camp:>15} ⚪ (sin campamentos)")

print("="*70)
print(f"\n{'TOTAL':<30} {rm_comunas['campamentos'].sum():>15}")
print("="*70)

╔══════════════════════════════════════════════════════════════╗
║  REGIÓN METROPOLITANA DE SANTIAGO - TODAS LAS COMUNAS        ║
╚══════════════════════════════════════════════════════════════╝

Total de comunas: 52
Comunas CON campamentos: 32
Comunas SIN campamentos (0): 20
Total de campamentos: 153

COMUNA                             CAMPAMENTOS
Lampa                                       22 🔴
Maipú                                       13 🔴
Puente Alto                                 12 🔴
Colina                                      11 🔴
Huechuraba                                  10 🔴
San José de Maipo                            9 🔴
La Florida                                   8 🔴
San Bernardo                                 8 🔴
Buin                                         8 🔴
Talagante                                    7 🔴
Tiltil                                       7 🔴
Cerrillos                                    5 🔴
Lo Barnechea                                 4 🔴
Estación Cen

# Verificación: Región de Valparaíso

In [ ]:
# Filtrar comunas de Valparaíso
valpo_comunas = comunas_gdf[comunas_gdf['Region'].str.contains('Valparaíso', na=False)].copy()

# Ordenar por cantidad de campamentos (descendente)
valpo_comunas_sorted = valpo_comunas.sort_values('campamentos', ascending=False)

# Mostrar información
print(f"╔══════════════════════════════════════════════════════════════╗")
print(f"║  REGIÓN DE VALPARAÍSO - TODAS LAS COMUNAS                    ║")
print(f"╚══════════════════════════════════════════════════════════════╝\n")

print(f"Total de comunas: {len(valpo_comunas)}")
print(f"Comunas CON campamentos: {len(valpo_comunas[valpo_comunas['campamentos'] > 0])}")
print(f"Comunas SIN campamentos (0): {len(valpo_comunas[valpo_comunas['campamentos'] == 0])}")
print(f"Total de campamentos: {valpo_comunas['campamentos'].sum()}\n")

print("="*70)
print(f"{'COMUNA':<30} {'CAMPAMENTOS':>15}")
print("="*70)

# Mostrar TODAS las comunas (con y sin campamentos)
for idx, row in valpo_comunas_sorted.iterrows():
    camp = row['campamentos']
    comuna = row['Comuna']
    
    # Marcar visualmente las que tienen campamentos
    if camp > 0:
        print(f"{comuna:<30} {camp:>15} 🔴")
    else:
        print(f"{comuna:<30} {camp:>15} ⚪ (sin campamentos)")

print("="*70)
print(f"\n{'TOTAL':<30} {valpo_comunas['campamentos'].sum():>15}")
print("="*70)

╔══════════════════════════════════════════════════════════════╗
║  REGIÓN DE VALPARAÍSO - TODAS LAS COMUNAS                    ║
╚══════════════════════════════════════════════════════════════╝

Total de comunas: 38
Comunas CON campamentos: 24
Comunas SIN campamentos (0): 14
Total de campamentos: 335

COMUNA                             CAMPAMENTOS
Viña del Mar                               115 🔴
Valparaíso                                  81 🔴
Quilpué                                     34 🔴
San Antonio                                 23 🔴
Villa Alemana                               17 🔴
Cartagena                                   13 🔴
Quintero                                     8 🔴
San Felipe                                   7 🔴
Limache                                      6 🔴
Puchuncaví                                   4 🔴
Los Andes                                    4 🔴
Quillota                                     3 🔴
Putaendo                                     3 🔴
El Quisco   

# Extraer polígonos de campamentos del KML original

In [ ]:
import geopandas as gpd

# Leer el KML original que tiene los polígonos
kml_path = "tmp_kmz_extract/doc.kml"
layer_name = "CNC_2024-2025"

gdf_poly = gpd.read_file(kml_path, layer=layer_name)

# Asegurar CRS WGS84
if gdf_poly.crs is None or gdf_poly.crs.to_epsg() != 4326:
    gdf_poly = gdf_poly.set_crs(4326, allow_override=True)

# Filtrar solo polígonos (excluir puntos si hay)
gdf_poly = gdf_poly[gdf_poly.geometry.geom_type.isin(['Polygon', 'MultiPolygon'])].copy()

# Arreglar geometrías inválidas
gdf_poly["geometry"] = gdf_poly.geometry.buffer(0)

print(f"Total de polígonos de campamentos: {len(gdf_poly)}")
print(f"Tipos de geometría: {gdf_poly.geom_type.value_counts().to_dict()}")

# Guardar los polígonos
gdf_poly.to_file("data/campamentos_poligonos.geojson", driver="GeoJSON")
print("✅ Guardado: data/campamentos_poligonos.geojson")

Total de polígonos de campamentos: 1428
Tipos de geometría: {'Polygon': 1417, 'MultiPolygon': 11}
✅ Guardado: data/campamentos_poligonos.geojson
